In [1]:
import pandas as pd
import numpy as np

dataset_path = 'dataset/50Ktrain.csv'
data = pd.read_csv(dataset_path)
# data = pd.read_csv(dataset_path)
print(data.shape)
# 篩選資料集，若有缺失值則整row刪除

keys_immutable = ['race', 'sex', 'native-country'] 

# choose data[0] as input feature, for all data points, if the immutable feature of data point isn't same as data[0], then drop it
for key in keys_immutable:
    data = data[data[key] == data.iloc[0][key]]

# drop 'education' and 'fnlwgt' features
data = data.drop(['education', 'fnlwgt', 'occupation', 'race', 'sex', 'relationship'], axis=1)

print(data.shape)

data = data.reset_index(drop=True)
print(data.head(3))

(32561, 15)
(17653, 9)
   age  workclass  education-num  marital-status  capital-gain  capital-loss  \
0   39          5             13               2          2174             0   
1   50          1             13               0             0             0   
2   38          0              9               1             0             0   

   hours-per-week  native-country  50K  
0              40               0    0  
1              13               0    0  
2              40               0    0  


C:\Users\user\AppData\Local\Temp\ipykernel_12404\1460382295.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
# data standardization
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
# standardization all columns except the last column
data.iloc[:, :-1] = scaler.fit_transform(data.iloc[:, :-1])
# covert to dataframe
df = pd.DataFrame(data)
print(df.head(3))
print()

# K-means clustering
from sklearn.cluster import KMeans
x = df.iloc[:, :-1] # 最後一項不要 (income) 其他去做kmeans
model = KMeans(n_clusters=100, n_init='auto', random_state=1).fit(x)

# get centers of each cluster
centers = model.cluster_centers_
# add column name to centers
centers = pd.DataFrame(centers, columns=df.columns[:-1])
print(centers.head(3))
# save centers to csv file
np.savetxt('centers.csv', centers, delimiter=',')



C:\Users\user\AppData\Local\Temp\ipykernel_12404\1285118727.py:6: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[-0.05861002  0.75852599 -0.13289512 ... -1.32145659  0.01567507
 -1.32145659]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  data.iloc[:, :-1] = scaler.fit_transform(data.iloc[:, :-1])
C:\Users\user\AppData\Local\Temp\ipykernel_12404\1285118727.py:6: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[ 1.91805278 -0.03476306 -0.52296703 ... -0.52296703 -0.52296703
 -0.52296703]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  data.iloc[:, :-1] = scaler.fit_transform(data.iloc[:, :-1])
C:\Users\user\AppData\Local\Temp\ipykernel_12404\1285118727.py:6: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of panda

        age  workclass  education-num  marital-status  capital-gain  \
0 -0.058610   1.918053       1.104109        1.283321      0.088997   
1  0.758526  -0.034763       1.104109       -0.694371     -0.163767   
2 -0.132895  -0.522967      -0.510674        0.294475     -0.163767   

   capital-loss  hours-per-week  native-country  50K  
0     -0.238769       -0.222405               0    0  
1     -0.238769       -2.418858               0    0  
2     -0.238769       -0.222405               0    0  

        age  workclass  education-num  marital-status  capital-gain  \
0 -1.416313   3.187383      -1.703130        1.283321     -0.161880   
1 -0.417916  -0.487569      -0.360660        0.294475     -0.131463   
2  0.107413  -0.408269       1.281638       -0.652673     -0.163767   

   capital-loss  hours-per-week  native-country  
0     -0.238769       -0.620395             0.0  
1     -0.238769       -0.223142             0.0  
2      4.215229        0.092214             0.0  


In [3]:
# read centers from csv file
centers = pd.read_csv('centers.csv', header=None)
print(centers.head(3))

          0         1         2         3         4         5         6    7
0 -1.416313  3.187383 -1.703130  1.283321 -0.161880 -0.238769 -0.620395  0.0
1 -0.417916 -0.487569 -0.360660  0.294475 -0.131463 -0.238769 -0.223142  0.0
2  0.107413 -0.408269  1.281638 -0.652673 -0.163767  4.215229  0.092214  0.0


In [ ]:
# # 透過Elbow法找出最佳K值(資料群數)
# Dist = []
# K = range(1,18)
# for k in K:
#     kmeanModel = KMeans(n_clusters=k, n_init='auto', random_state=1).fit(x)
#     kmeanModel.fit(x)
#     Dist.append(kmeanModel.inertia_)
# print(Dist)
# plt.plot(range(1,18), Dist, 'bx-')
# plt.show()



# KNN

In [125]:
import pandas as pd
import queue
from sklearn.neighbors import NearestNeighbors
import numpy as np

q = queue.Queue()
q.put(1)

while not q.empty():
    index = q.get()

    df = pd.DataFrame(data)

    # Extract features for KNN
    features = df[[0, 1, 2, 4]]    # age education-num hours-per-week

    # choose data[0] as target data point for which you want to find neighbors
    target_data_point = features.iloc[index]

    # Create a KNN model
    knn_model = NearestNeighbors(n_neighbors=6, algorithm='auto')  # Adjust n_neighbors as needed

    # Fit the model with your dataset
    knn_model.fit(features)

    # Find the indices of the nearest neighbors (excluding the target_data_point itself)
    _, indices = knn_model.kneighbors([target_data_point])
    neighbor_indices: np.ndarray = indices[0]
    neighbor_indices = np.delete(neighbor_indices, np.where(neighbor_indices == index))

    # for i in neighbor_indices:
    #     q.put(i)

    # Print the neighbors
    print("Nearest neighbors:")
    print(df.iloc[neighbor_indices])

    # store the index of the nearest neighbors
    neighbor_indices = neighbor_indices.tolist()

Nearest neighbors:
              0         1         2         3         4        5
15081  0.810470  1.097452 -0.165421 -0.241688 -2.961453 -0.70739
10727  1.197121  1.097452 -0.165421 -0.241688 -2.372177 -0.70739
8454   0.423820  1.097452 -0.165421 -0.241688 -2.372177 -0.70739
4792   0.887800  1.097452 -0.165421 -0.241688 -2.961453 -0.70739
2774   0.346489  1.097452 -0.165421 -0.241688 -2.624724 -0.70739


# generate a cost function that takes in two data point and returns a cost table with two columns: payment and time
payment: sum all
time: pick maxium

In [19]:
# generate a cost function that takes in two data point and returns a cost table with two columns: payment and time
def generate_cost_table(data_point_1, data_point_2):
    # calculate the cost of payment
    totaldiscrete = 0
    # discrete features
    if data_point_1['relationship'] != data_point_2['relationship']:
        # use Euclidean distance to calculate the cost of payment
        totaldiscrete += np.linalg.norm(data_point_1['relationship'] - data_point_2['relationship'])
    
    if data_point_1['workclass'] != data_point_2['workclass']:
        totaldiscrete += np.linalg.norm(data_point_1['workclass'] - data_point_2['workclass'])
    if data_point_1['education'] != data_point_2['education']:
        totaldiscrete += np.linalg.norm(data_point_1['education'] - data_point_2['education'])

    # calculate the cost of time
    totalTime = 0
    totalPay = 0
    if data_point_1['age'] != data_point_2['age']:
        totalTime = max(np.abs(data_point_1['age'] - data_point_2['age']), totalTime)
    if data_point_1['education-num'] != data_point_2['education-num']:
        totalTime = max(np.abs(data_point_1['education-num'] - data_point_2['education-num']), totalTime)
    if data_point_1['capital-gain'] != data_point_2['capital-gain']:
        totalPay += (np.abs(data_point_1['capital-gain'] - data_point_2['capital-gain']), totalPay)
    if data_point_1['capital-loss'] != data_point_2['capital-loss']:
        totalPay -= (np.abs(data_point_1['capital-loss'] - data_point_2['capital-loss']), totalPay)
    if data_point_1['hours-per-week'] != data_point_2['hours-per-week']:
        totalTime = max(np.abs(data_point_1['hours-per-week'] - data_point_2['hours-per-week']), totalTime)
        totalPay += np.abs(data_point_1['hours-per-week'] - data_point_2['hours-per-week'])

    #　now we have totaldiscrete、totalTime、totalPay => use a non-linear function to combine them, and then find the parato front
    return totaldiscrete, totalTime, totalPay

# generate a cost function, and then return 5 point in the parato front( you can change it into "dominated point")
def generate_cost_table(totaldiscrete, totalTime, totalPay):
    # a function: f(totalPay) = totaldiscrete/totalTime
    def f(p):
        return 1 / p
    table = []
    while len(table) < 5:
        p = np.random.uniform(0, totalPay)
        t = f(p)
        if t > totalTime:
            continue
        table.append([p, t])
    print(table)

In [ ]:
from pathlib import Path
from typing import Literal

import pandas as pd

import torch as pt
from torch import nn


continuous = ['age', 'fnlwgt', 'education-num',
              'capital-gain', 'capital-loss', 'hours-per-week']
categorical = ['workclass', 'marital-status', 'occupation',
               'relationship', 'race', 'sex', 'native-country']
target = ['50K']


class MLP50K(nn.Module):
    def __init__(self):
        super().__init__()
        self.n_continuous = 6
        self.n_categorical = 9, 7, 15, 6, 5, 2, 42
        self.n_features = self.n_continuous + sum(self.n_categorical)
        self.linear = nn.Sequential(
            nn.Linear(self.n_features, 64),
            nn.ReLU(True),
            nn.Linear(64, 32),
            nn.ReLU(True),
            nn.Linear(32, 1),
            nn.Sigmoid()
        )

    def forward(self, x1: pt.Tensor[pt.float], x2: pt.Tensor[pt.int]) -> pt.Tensor:
        a = [x1]
        for i, n in enumerate(self.n_categorical):
            x = nn.functional.one_hot(x2[i], n)
            a.append(x)
        a = pt.cat(a, 1)
        x = self.linear(a)
        return x


def load_model() -> MLP50K:
    path = Path('resource/model.pt')
    state = pt.load(path)
    model = MLP50K()
    model.load_state_dict(state)
    model.eval()
    return model


def load_dataset(which: Literal['train', 'test']) -> pd.DataFrame:
    path = Path(f'resource/50K.{which}.csv')
    return pd.read_csv(path)
